#Tutorial 10 - BERT

In [1]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 39.0 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 636 kB 42.9 MB/s 
     |████████████████████████████████| 3.3 MB 38.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


##Defining The Model

In [2]:
from torch import nn
from transformers import BertModel

In [11]:
class RumorDetectionModel(nn.Module):

    def __init__(self, num_classes):
        super(RumorDetectionModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.bert: BertModel
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, token_type_ids, labels):
        x = self.bert(input_ids, attention_mask, token_type_ids)
        x = x.last_hidden_state
        x = x.mean(axis=1)
        x = self.classifier(x)
        loss = self.loss(x, labels)
        return x, loss

##Creating The Dataset

In [4]:
DATA_PATH = "/content/drive/MyDrive/Technion/NLP course Winter 2021-2022/Tutorials/Tutorial_10_bert/rumor data"

In [5]:
from transformers import BertTokenizerFast
from torch.utils.data import Dataset
import pandas as pd


In [6]:
class RumorDataSet(Dataset):

    def __init__(self, file_path):
        data = pd.read_csv(filepath_or_buffer=file_path)
        self.text = data['text'].tolist()
        self.labels = data['labels'].tolist()
        self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
        tokenized = self.tokenizer.batch_encode_plus(self.text, max_length=128, padding="max_length",
                                        return_tensors="pt", truncation=True, add_special_tokens=True)
        self.input_ids = tokenized.data['input_ids']
        self.attention_mask = tokenized['attention_mask']
        self.token_type_ids = tokenized['token_type_ids']
    
    def __getitem__(self, item):
        return {'input_ids': self.input_ids[item],
                "attention_mask": self.attention_mask[item], 
                "token_type_ids": self.token_type_ids[item],
                'labels': self.labels[item]}
    
    def __len__(self):
        return len(self.input_ids)

In [7]:
train_dataset = RumorDataSet(f"{DATA_PATH}/train.csv")
test_dataset = RumorDataSet(f"{DATA_PATH}/test.csv")

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

##Training The Model

In [14]:
import torch
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm

In [16]:
BATCH_SIZE = 32
NUM_EPOCHS = 5

In [19]:
model = RumorDetectionModel(2)
optimizer = Adam(params=model.parameters(), lr=3e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_loaders = {"train": DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True),
                "test": DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)}
data_sets = {'train': train_dataset, 'test': test_dataset}
model.to(device)

best_f1 = 0.0

for epoch in range(NUM_EPOCHS):
    print(f'Epoch {epoch + 1}/{NUM_EPOCHS}')
    print('-' * 10)

    for phase in ['train', 'test']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        predictions, truth = [], []

        for batch in tqdm(data_loaders[phase], total=len(data_loaders[phase])):
            batch_size = 0
            for k, v in batch.items():
                batch[k] = v.to(device)
                batch_size = v.shape[0]

            optimizer.zero_grad()
            if phase == 'train':
                outputs, loss = model(**batch)
                loss.backward()
                optimizer.step()
            else:
                with torch.no_grad():
                    outputs, loss = model(**batch)
            pred = outputs.argmax(dim=-1).clone().detach().cpu().view(-1).tolist()
            cur_truth = batch['labels'].clone().detach().cpu().view(-1).tolist()

            running_loss += loss.item() * batch_size
            predictions += pred
            truth += cur_truth

        epoch_loss = running_loss / len(data_sets[phase])
        epoch_f1 = f1_score(predictions, truth)

        epoch_f1 = round(epoch_f1, 5)
        if phase.title() == "test":
            print(f'{phase.title()} Loss: {epoch_loss:.4e} F1: {epoch_f1}')
        else:
            print(f'{phase.title()} Loss: {epoch_loss:.4e} F1: {epoch_f1}')
        if phase == 'test' and epoch_f1 > best_f1:
            best_f1 = epoch_f1
    print()

print(f'Best Validation F1: {best_f1:4f}')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/5
----------


100%|██████████| 145/145 [03:12<00:00,  1.33s/it]


Train Loss: 4.0615e-01 F1: 0.71642


100%|██████████| 182/182 [01:26<00:00,  2.11it/s]


...

Train Loss: 1.0563e-01 F1: 0.94357


100%|██████████| 182/182 [01:26<00:00,  2.11it/s]


Test Loss: 1.4586e-01 F1: 0.92912

Epoch 4/5
----------


 81%|████████▏ | 118/145 [02:37<00:36,  1.34s/it]


KeyboardInterrupt: ignored